## 連接ES

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import random
import string

es_kwargs = {
    "hosts": "https://localhost:9200",
    "basic_auth": ('elastic', 'pass.123')
}

es = Elasticsearch(**es_kwargs, ca_certs='../ca.crt')
es_info = es.info()
print(f"Connected to cluster named '{es_info['cluster_name']}' (version: {es_info['version']['number']})")


Connected to cluster named 'docker-cluster' (version: 8.15.0)


## 產生資料的函數

In [2]:
import random
import string
from datetime import datetime, timedelta

def generate_random_data(n):
    data = []
    # 設定時間範圍為2024整年
    end_date = datetime(2024, 12, 31)
    start_date = end_date - timedelta(days=365)

    for i in range(n):
        random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
        random_date = start_date + timedelta(
            seconds=random.randint(0, int((end_date - start_date).total_seconds()))
        )
        data.append({
            "id": i,
            "random_string": random_string,
            "random_number": random.randint(1, 1000),
            "timestamp": random_date.isoformat()  # 使用 ISO 格式儲存時間
        })
    return data

## 建立索引並插入資料

In [3]:
from eland import pandas_to_eland

index_name = "random_data_index"

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

num_records = 20000
random_data = generate_random_data(num_records)

df = pd.DataFrame(random_data)

ed = pandas_to_eland(
    pd_df=df,
    es_client=es,
    es_dest_index=index_name,
    es_if_exists="replace",
    es_refresh=True
)



## 驗證資料是否插入成功

In [4]:
print(f"成功插入 {num_records} 筆資料到Index '{index_name}'")
count = es.count(index=index_name)
print(f"Index '{index_name}' 中的documnet數量: {count['count']}")

成功插入 20000 筆資料到Index 'random_data_index'
Index 'random_data_index' 中的documnet數量: 20000


## 驗證eland是否可以讀出超過一萬筆資料

In [18]:
import eland

# 從ES中讀取資料
ed_df = eland.DataFrame(es_client=es, es_index_pattern=index_name)

total_rows = len(ed_df)
print(f"Index '{index_name}' 中的總資料筆數: {total_rows}")

# 把資料轉換成Pandas DataFrame
pd_df = ed_df.to_pandas()

print(f"成功讀取的筆數: {len(pd_df)}")
print("\n前5筆資料:")
print(pd_df.head())
print("\n資料類型:")
print(pd_df.dtypes)



Index 'random_data_index' 中的總資料筆數: 20000
成功讀取的筆數: 20000

前5筆資料:
        id  random_number random_string                   timestamp
3387  3387            746    hiczUfNgGJ  2024-07-24T19:50:33.686978
5054  5054            121    Ovp0qVvh65  2024-01-30T10:45:06.686978
3393  3393            981    blzUATWSD1  2024-03-03T15:39:53.686978
3398  3398            215    wpnwrIoLUy  2024-06-28T09:14:46.686978
5062  5062            418    ZiPwXaThbA  2024-07-17T11:26:12.686978

資料類型:
id                int64
random_number     int64
random_string    object
timestamp        object
dtype: object


## 從ES中讀取資料

In [24]:
# 從ES中讀取資料
ed_df = eland.DataFrame(es_client=es, es_index_pattern=index_name)

# 詳細的API說明可參考官方文件:
# https://eland.readthedocs.io/en/v8.15.0/reference/supported_apis.html

# 秀出ed_df的物件類型
print(f"ed_df的物件類型: {type(ed_df)}")

ed_df的物件類型: <class 'eland.dataframe.DataFrame'>


## 查詢特定時間範圍內的數據

In [25]:

start_time = "2024-01-01T00:00:00"
end_time = "2024-06-30T23:59:59"

time_range_query = ed_df.es_query({
    "query": {
        "range": {
            "timestamp": {
                "gte": start_time,
                "lte": end_time
            }
        }
    }
})

print(f"在 {start_time} 到 {end_time} 之間的記錄數：{time_range_query.shape[0]}")


在 2024-01-01T00:00:00 到 2024-06-30T23:59:59 之間的記錄數：9845


## 查詢八月的資料

In [26]:

august_data = ed_df.es_query({
    "query": {
        "range": {
            "timestamp": {
                "gte": "2024-08-01T00:00:00",
                "lte": "2024-08-31T23:59:59"
            }
        }
    }
})

print(f"在2024年8月的記錄數：{august_data.shape[0]}")


在2024年8月的記錄數：1361


## 查找特定日期的資料

In [27]:
specific_date = "2024-05-15"
specific_date_data = ed_df.es_query({
    "query": {
        "range": {
            "timestamp": {
                "gte": specific_date,
                "lt": f"{specific_date}||+1d"
            }
        }
    }
})

print(f"{specific_date} 的記錄數：{specific_date_data.shape[0]}")

2024-05-15 的記錄數：52


## 時間範圍和其他條件組合查詢

In [28]:

combined_query = ed_df.es_query({
    "query": {
        "bool": {
            "must": [
                {
                    "range": {
                        "timestamp": {
                            "gte": "2024-01-01",
                            "lt": "2025-01-01"
                        }
                    }
                },
                {
                    "range": {
                        "random_number": {
                            "gt": 500
                        }
                    }
                }
            ]
        }
    }
})

print(f"2024 年內 random_number 大於 500 的記錄數：{combined_query.shape[0]}")

2024 年內 random_number 大於 500 的記錄數：6447


## 基本篩選查詢

In [29]:
filtered_df = ed_df[ed_df['random_number'] > 500]
print(f"隨機數大於500的資料數量: {len(filtered_df)}")

隨機數大於500的資料數量: 9910


## 排序並獲取前10筆記錄（eland不支援pandas的sort_values方法）

In [30]:
sorted_df = ed_df.es_query({
    "sort": [{"random_number": {"order": "desc"}}]
})
pandas_df = ed_df.to_pandas()
sorted_df = pandas_df.sort_values('random_number', ascending=True)
print(f"隨機數最大的10筆記錄:")
print(sorted_df.head(10))

隨機數最大的10筆記錄:
          id  random_number random_string                   timestamp
7257    7257              1    IhkdVfe5yx  2024-06-10T03:18:27.686978
5517    5517              1    iAGWdyQvTi  2023-10-30T11:02:31.686978
17585  17585              1    390ADMKp6n  2024-02-03T08:17:31.686978
18750  18750              1    NGUq7qOZed  2024-05-17T21:25:29.686978
15061  15061              1    qUtpywF5D0  2024-06-24T09:31:43.686978
16806  16806              1    NXCG7s85aP  2023-10-30T05:22:36.686978
9750    9750              1    ghR344RSBz  2024-07-21T17:34:26.686978
9340    9340              1    Vh4nsBowEW  2024-07-15T23:43:11.686978
13609  13609              1    i8R04yypa7  2024-02-19T20:40:40.686978
14128  14128              1    VFE7Z7Po6d  2024-04-19T01:23:33.686978


## 分組聚合

In [31]:
grouping_df = ed_df.groupby('random_number').agg(['count'])
grouping_df = grouping_df.sort_values('random_number', ascending=False)
print("\n出現次數最多的5個隨機數:")
print(grouping_df.head(5))


出現次數最多的5個隨機數:
                 id random_string timestamp
              count         count     count
random_number                              
1000             20            20        20
999              21            21        21
998              17            17        17
997              14            14        14
996              29            29        29


## 複雜條件查詢

In [32]:
complex_query = ed_df[(ed_df['random_number'] > 300) & (ed_df['random_number'] < 700)]
print(f"\n隨機數在300到700之間的資料數量: {len(complex_query)}")


隨機數在300到700之間的資料數量: 7953


## 文字搜尋

In [33]:
char_query = ed_df['random_string'].to_pandas()
char_query = char_query[char_query.str.contains('a')]
print(f"\n隨機字串中包含'a'的資料數量: {len(char_query)}")


隨機字串中包含'a'的資料數量: 3001


## 獲取特定ID範圍的資料

In [34]:
id_range_query = ed_df[(ed_df['id'] >= 1000) & (ed_df['id'] < 1010)]
print("\nID從1000到1009的資料:")
print(id_range_query.to_pandas())


ID從1000到1009的資料:
        id  random_number random_string                   timestamp
1000  1000            115    NTldRWkdDQ  2023-11-07T09:30:29.686978
1001  1001            367    UJHtqclM1h  2024-07-31T14:23:34.686978
1002  1002            894    AzMBMkJCm8  2023-10-19T07:13:31.686978
1003  1003            827    X0KY4EXtpt  2024-08-20T10:48:35.686978
1004  1004            870    ghPyYS1ufc  2023-10-01T14:18:11.686978
1005  1005            282    EIiIGf3njF  2024-07-25T07:36:51.686978
1006  1006             17    UOPrRAUE1j  2024-01-05T17:49:59.686978
1007  1007            611    oGNyfStGlE  2023-08-28T11:13:17.686978
1008  1008            884    SIZMK5crCJ  2024-05-08T17:15:18.686978
1009  1009             80    2tdAXnMKSS  2024-04-18T06:45:50.686978


## 計算統計資訊

In [35]:
statics = ed_df['random_number'].describe()
print("\n隨機數的統計資訊:")
print(statics)


隨機數的統計資訊:
count    20000.000000
mean       500.331300
std        288.927971
min          1.000000
25%        250.491300
50%        497.951880
75%        749.803185
max       1000.000000
Name: random_number, dtype: float64


## 使用ES原生查詢

In [36]:
original_query = ed_df.es_query(
    {
        "query": {
            "range": {
                "random_number": {
                    "gte": 990
                }
            }
        }
    }
)
print(f"\n使用ES原生查詢，隨機數大於等於990的資料數量: {len(original_query)}")


使用ES原生查詢，隨機數大於等於990的資料數量: 223
